# Zugriffsdaten-Analyse

## Setup

In [1]:
import pyspark
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.appName("main") \
  .master("spark://localhost:7077") \
  .config("spark.executor.memory", "5G") \
  .getOrCreate()

22/06/26 16:22:29 WARN Utils: Your hostname, Pauls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.12 instead (on interface en0)
22/06/26 16:22:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/26 16:22:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Datenimport

In [24]:
raw_df = spark.read.csv("/tmp/data/access.log", sep=" ")

In [25]:
raw_df.show(5)

+-------------+---+---+--------------------+------+--------------------+---+-----+--------------------+--------------------+----+
|          _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6|  _c7|                 _c8|                 _c9|_c10|
+-------------+---+---+--------------------+------+--------------------+---+-----+--------------------+--------------------+----+
| 54.36.149.41|  -|  -|[22/Jan/2019:03:5...|+0330]|GET /filter/27|13...|200|30577|                   -|Mozilla/5.0 (comp...|   -|
|  31.56.96.51|  -|  -|[22/Jan/2019:03:5...|+0330]|GET /image/60844/...|200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|   -|
|  31.56.96.51|  -|  -|[22/Jan/2019:03:5...|+0330]|GET /image/61474/...|200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|   -|
|40.77.167.129|  -|  -|[22/Jan/2019:03:5...|+0330]|GET /image/14925/...|200| 1696|                   -|Mozilla/5.0 (comp...|   -|
|  91.99.72.15|  -|  -|[22/Jan/2019:03:5...|+0330]|GET /product/3189...|200|41483|        

In [29]:
df = raw_df.withColumnRenamed("_c0", "ip") \
  .drop("_c1") \
  .withColumnRenamed("_c2", "user") \
  .withColumn("user", regexp_replace("user", "-", "")) \
  .withColumnRenamed("_c3", "timestamp") \
  .withColumn("timestamp", regexp_replace("timestamp", "^\[", "")) \
  .withColumnRenamed("_c4", "timezone") \
  .withColumn("timezone", regexp_replace("timezone", "\]$", "")) \
  .withColumnRenamed("_c5", "request_line") \
  .withColumnRenamed("_c6", "status") \
  .withColumnRenamed("_c7", "bytes_sent") \
  .withColumnRenamed("_c8", "referer") \
  .withColumn("referer", regexp_replace("referer", "-", "")) \
  .withColumnRenamed("_c9", "user_agent") \
  .drop("_c10")

df.show(5)

+-------------+----+--------------------+--------+--------------------+------+----------+--------------------+--------------------+
|           ip|user|           timestamp|timezone|        request_line|status|bytes_sent|             referer|          user_agent|
+-------------+----+--------------------+--------+--------------------+------+----------+--------------------+--------------------+
| 54.36.149.41|    |22/Jan/2019:03:56:14|   +0330|GET /filter/27|13...|   200|     30577|                    |Mozilla/5.0 (comp...|
|  31.56.96.51|    |22/Jan/2019:03:56:16|   +0330|GET /image/60844/...|   200|      5667|https://www.zanbi...|Mozilla/5.0 (Linu...|
|  31.56.96.51|    |22/Jan/2019:03:56:16|   +0330|GET /image/61474/...|   200|      5379|https://www.zanbi...|Mozilla/5.0 (Linu...|
|40.77.167.129|    |22/Jan/2019:03:56:17|   +0330|GET /image/14925/...|   200|      1696|                    |Mozilla/5.0 (comp...|
|  91.99.72.15|    |22/Jan/2019:03:56:17|   +0330|GET /product/3189...|   20

In [35]:
df = df.withColumn("product_id", regexp_extract("request_line", "^GET \/product\/(\d+)", 1))
df.show(5)

+-------------+----+--------------------+--------+--------------------+------+----------+--------------------+--------------------+----------+
|           ip|user|           timestamp|timezone|        request_line|status|bytes_sent|             referer|          user_agent|product_id|
+-------------+----+--------------------+--------+--------------------+------+----------+--------------------+--------------------+----------+
| 54.36.149.41|    |22/Jan/2019:03:56:14|   +0330|GET /filter/27|13...|   200|     30577|                    |Mozilla/5.0 (comp...|          |
|  31.56.96.51|    |22/Jan/2019:03:56:16|   +0330|GET /image/60844/...|   200|      5667|https://www.zanbi...|Mozilla/5.0 (Linu...|          |
|  31.56.96.51|    |22/Jan/2019:03:56:16|   +0330|GET /image/61474/...|   200|      5379|https://www.zanbi...|Mozilla/5.0 (Linu...|          |
|40.77.167.129|    |22/Jan/2019:03:56:17|   +0330|GET /image/14925/...|   200|      1696|                    |Mozilla/5.0 (comp...|          |

In [43]:
product_group_df = df.filter(df.product_id != "").groupBy("product_id").count().sort(col("count").desc())

+----------+-----+
|product_id|count|
+----------+-----+
|     33953|  764|
|     34286|  738|
|     33968|  692|
|     33954|  687|
|     33956|  594|
|     33960|  533|
|     33952|  517|
|     31720|  483|
|     33722|  347|
|     33626|  323|
|     33978|  320|
|     34290|  286|
|     33639|  275|
|     34288|  269|
|     31577|  264|
|       278|  259|
|     33598|  243|
|     14233|  243|
|     30177|  243|
|      8972|  241|
+----------+-----+
only showing top 20 rows



In [45]:
pd_df = product_group_df.toPandas()

In [49]:
pd_df.describe()

,count
count,16823.000000
mean,13.188016
std,24.351003
min,1.000000
25%,2.000000
50%,6.000000
75%,16.000000
max,764.000000


In [47]:
pd_df.head()

,product_id,count
0,33953,764
1,34286,738
2,33968,692
3,33954,687
4,33956,594
